In [ ]:
import sys, os
import rasterio, imageio

import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio.plot

from shapely.geometry import Point, box
from space2stats_client import Space2StatsClient
from rasterio.windows import from_bounds

sys.path.insert(0, r"C:\WBG\Work\Code\GOSTrocks\src")

import GOSTrocks.ntlMisc as ntlMisc
import GOSTrocks.mapMisc as mapMisc
import GOSTrocks.dataMisc as dMisc

# Make jupyter notebook autoreload modules
%load_ext autoreload
%autoreload 2

# Mute InsecureRequestWarning
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
iso3 = 'MOZ'
out_folder = "C:/WBG/Work/Projects/NTL_animation/{}".format(iso3)
ntl_data_folder = os.path.join(out_folder, "ntl_data")
map_folder = os.path.join(out_folder, "maps")
if not os.path.exists(map_folder):
    os.makedirs(map_folder)
admin0_bounds_file = r"C:\WBG\Work\data\ADMIN\BETTER_FORMATS\WB_GAD_ADM0_complete.gpkg"
ocean_mask = r"C:\WBG\Work\data\ADMIN\BETTER_FORMATS\ocean_mask.gpkg"

all_Admin = gpd.read_file(admin0_bounds_file)
admin_bounds = all_Admin[all_Admin['ISO_A3'] == iso3]
other_admin_bounds = all_Admin[all_Admin['ISO_A3'] != iso3]
ocean_mask = gpd.read_file(ocean_mask)

'''
s2s_client = Space2StatsClient(verify_ssl=False)
admin_bounds = s2s_client.fetch_admin_boundaries(iso3, "ADM2")
'''
all_ntl_files = dMisc.aws_search_ntl()
# Enable if you want to download the NLT monthly files directly
# ntl_data = ntlMisc.extract_monthly_ntl(admin_bounds, ntl_data_folder, sel_files=all_ntl_files[:2])

In [ ]:
# Turn feature points into a geodataframe
sel_points = [
    [32.44409468679876,-25.778349011951352],
    [32.45707495887211,-25.798569132332307],
    [32.47687067993886,-25.881611483098663]
]
sel_points_geom = [Point(x) for x in sel_points]
sel_points = gpd.GeoDataFrame(geometry=sel_points_geom, crs=4326)
sel_points.to_file("C:/WBG/Work/Temp/sel_points.gpkg", driver='GPKG')

In [ ]:
def map_ntl(sel_image, b, admin, ocean, out_file=''):
    year = os.path.basename(sel_image).split('_')[2][:4]
    with rasterio.Env(GDAL_HTTP_UNSAFESSL = 'YES'): 
        selR = rasterio.open(sel_image)
        #Set extent to the whole country
        sel_window = from_bounds(b[0], b[1], b[2], b[3], selR.transform)
        sel_transform = selR.window_transform(sel_window)
        
        data = selR.read(1, window=sel_window)
            
        fig, ax = plt.subplots(figsize=(10, 8))
        rasterio.plot.show(data, transform=sel_transform, ax=ax, cmap='magma', vmin=0, vmax=50)

        admin.plot(ax=ax, facecolor='black', edgecolor='grey') # Adjust colors as needed
        ocean.plot(ax=ax, facecolor='#000435', edgecolor='grey') # Ocean mask
        sel_points.plot(ax=ax, color='#e1e8e3', marker='x', markersize=50, label='Selected Points') # Plot selected points

        ax.set_xlim(b[0], b[2])
        ax.set_ylim(b[1], b[3])
        plt.text(b[2], b[3], year, fontsize=12, ha='right', va='top', color='white')

        if out_file != '':
            plt.savefig(out_map, dpi=300, bbox_inches='tight')
        else:
            plt.show()

sel_box = box(*sel_points.total_bounds).buffer(0.1)
map_ntl(all_ntl_files[0], sel_box.bounds, other_admin_bounds, ocean_mask)

In [25]:
for sel_image in all_ntl_files:
    # Open with rasterio and set environment to ignore ssl errors
    out_map = os.path.join(map_folder, "ntl_{}.png".format(sel_image.split('/')[-1].split('.')[0]))    
    if not os.path.exists(out_map):
        map_ntl(sel_image, sel_box.bounds, other_admin_bounds, ocean_mask, out_map)

# Trun produced images into an animated GIF

In [26]:
images = [os.path.join(map_folder, f) for f in os.listdir(map_folder) if 'npp' in f and f.endswith('.png')]
images.sort()  # Sort images by name to ensure chronological order
all_images = []
for f in images:
    all_images.append(imageio.imread(f))
imageio.mimsave(os.path.join(map_folder, "ntl_animation.gif"), all_images, duration=2, loop=0)

C:\Users\WB411133\AppData\Local\Temp\ipykernel_31692\1055495744.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  all_images.append(imageio.imread(f))
